In [2]:
# Imports
import tfx
from tfx.proto import example_gen_pb2
from tfx.components import FileBasedExampleGen, StatisticsGen
from tfx.components.example_gen.custom_executors import parquet_executor
from tfx.dsl.components.base import executor_spec
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.types import standard_component_specs

from tensorflow.core.example.example_pb2 import Example
from tensorflow.python.data import TFRecordDataset
import os

import pprint
pp = pprint.PrettyPrinter()

In [3]:


output_config = example_gen_pb2.Output(
    split_config=example_gen_pb2.SplitConfig(splits=[
        example_gen_pb2.SplitConfig.Split(name='train', hash_buckets=2)
        ,example_gen_pb2.SplitConfig.Split(name='eval', hash_buckets=1)
    ]))
input_config = example_gen_pb2.Input(splits=[
    example_gen_pb2.Input.Split(name='parquet',
                                pattern='*.parquet'),
])

example_gen = FileBasedExampleGen(
    custom_executor_spec=executor_spec.ExecutorClassSpec(
        parquet_executor.Executor),
    input_base="/home/mlops/project/DeltaLake/platinum_data/tfx_test",
    input_config=input_config,
    output_config=output_config).with_id('ParquetExampleGen')


context = InteractiveContext(
  pipeline_name='ConcertsPipe',
  pipeline_root='/home/mlops/project/TFX'
  )
context.run(example_gen)


artifact = example_gen.outputs['examples'].get()[0]
print(artifact.split_names, artifact.uri)

["train", "eval"] /home/mlops/project/TFX/ParquetExampleGen/examples/272


In [4]:

train_uri = os.path.join(example_gen.outputs['examples'].get()[0].uri, 'Split-train')

# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Create a `TFRecordDataset` to read these files
dataset = TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

# Iterate over the first 3 records and decode them.
for tfrecord in dataset.take(3):
  serialized_example = tfrecord.numpy()
  example = Example()
  example.ParseFromString(serialized_example)
  pp.pprint(example)



features {
  feature {
    key: "spotify_followers"
    value {
      float_list {
        value: 0.0011630194494500756
      }
    }
  }
  feature {
    key: "spotify_genres_9"
    value {
      int64_list {
        value: 0
      }
    }
  }
  feature {
    key: "spotify_popularity"
    value {
      float_list {
        value: 0.4683544337749481
      }
    }
  }
  feature {
    key: "ticketmaster_price_max"
    value {
      float_list {
        value: 0.07500000298023224
      }
    }
  }
}

features {
  feature {
    key: "spotify_followers"
    value {
      float_list {
        value: 0.013742172159254551
      }
    }
  }
  feature {
    key: "spotify_genres_9"
    value {
      int64_list {
        value: 1
      }
    }
  }
  feature {
    key: "spotify_popularity"
    value {
      float_list {
        value: 0.7215189933776855
      }
    }
  }
  feature {
    key: "ticketmaster_price_max"
    value {
      float_list {
        value: 0.05624999850988388
      }
    }
  }


In [5]:

statistics_gen = StatisticsGen(examples=example_gen.outputs['examples'])
context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 848
    outputs:
        statistics: Channel(
            type_name: ExampleStatistics
            artifacts: [Artifact(artifact: id: 141
        type_id: 16
        uri: "/home/mlops/project/TFX/StatisticsGen/statistics/273"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "statistics"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "StatisticsGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.5.0"
          }
        }
        state: LIVE
        create_time_since_epoch: 1643224502755
        last_update_time_since_epoch: 1643224506781
        , artifact_type: id: 16
        name: "ExampleStatistics"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        base_type: STATISTICS
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [6]:
context.show(statistics_gen.outputs['statistics'])

In [7]:
schema_gen = tfx.components.SchemaGen(
    statistics=statistics_gen.outputs['statistics'],
    infer_feature_shape=False)
context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 849
    outputs:
        schema: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 142
        type_id: 18
        uri: "/home/mlops/project/TFX/SchemaGen/schema/274"
        custom_properties {
          key: "name"
          value {
            string_value: "schema"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "SchemaGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.5.0"
          }
        }
        state: LIVE
        create_time_since_epoch: 1643224506852
        last_update_time_since_epoch: 1643224506867
        , artifact_type: id: 18
        name: "Schema"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [8]:
context.show(schema_gen.outputs['schema'])

schema_gen.outputs['schema'].get()[0]

,Type,Presence,Valency,Domain
Feature name,,,,
'spotify_followers',FLOAT,required,single,-
'spotify_genres_9',INT,required,single,-
'spotify_popularity',FLOAT,required,single,-
'ticketmaster_price_max',FLOAT,required,single,-


.type,<class 'tfx.types.standard_artifacts.Schema'>
.uri,/home/mlops/project/TFX/SchemaGen/schema/274


In [9]:
# for manual Schema
# from tfx.components.schema_gen.import_schema_gen.component import ImportSchemaGen
# schema_gen = ImportSchemaGen(
#     schema_file='/home/mlops/project/TFX/SchemaGen/schema/34/schema.pbtxt')
# context.run(schema_gen)

example_validator = tfx.components.ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema'])
context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 850
    outputs:
        anomalies: Channel(
            type_name: ExampleAnomalies
            artifacts: [Artifact(artifact: id: 143
        type_id: 20
        uri: "/home/mlops/project/TFX/ExampleValidator/anomalies/275"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "anomalies"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "ExampleValidator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.5.0"
          }
        }
        state: LIVE
        create_time_since_epoch: 1643224506944
        last_update_time_since_epoch: 1643224506958
        , artifact_type: id: 20
        name: "ExampleAnomalies"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [10]:
context.show(example_validator.outputs['anomalies'])

In [11]:
def func():
    print("some")

In [12]:
# transform = tfx.components.Transform(
#     examples=example_gen.outputs['examples'],
#     schema=schema_gen.outputs['schema']
#
# )
# context.run(transform)

In [13]:
_trainer_module_file = '/home/mlops/project/test/penguin_trainer.py'

In [14]:
%%writefile {_trainer_module_file}

from typing import List
from absl import logging
import tensorflow as tf
from tensorflow import keras
from tensorflow_transform.tf_metadata import schema_utils

from tfx import v1 as tfx
from tfx_bsl.public import tfxio
from tensorflow_metadata.proto.v0 import schema_pb2


_FEATURE_KEYS = [
    'spotify_popularity', 'ticketmaster_price_max'
    # 'spotify_genres', 'spotify_genres_18', 'spotify_genres_9', 'spotify_href', 'spotify_id', 'spotify_image_url', 'spotify_type', 'spotify_uri', 'spotify_url', 'ticketmaster_artist_locale', 'ticketmaster_date', 'ticketmaster_distance', 'ticketmaster_id', 'ticketmaster_locale', 'ticketmaster_name', 'ticketmaster_price_min', 'ticketmaster_units', 'ticketmaster_url', 'ticketmaster_venue_address', 'ticketmaster_venue_city', 'ticketmaster_venue_country', 'ticketmaster_venue_id', 'ticketmaster_venue_latitude', 'ticketmaster_venue_longitude', 'ticketmaster_venue_name', 'ticketmaster_venue_postal_code', 'ticketmaster_venue_timezone', 'ticketmaster_venue_url', 'spotify_followers'
]
_LABEL_KEY = 'spotify_genres_9'

_TRAIN_BATCH_SIZE = 20
_EVAL_BATCH_SIZE = 10

# Since we're not generating or creating a schema, we will instead create
# a feature spec.  Since there are a fairly small number of features this is
# manageable for this dataset.
_FEATURE_SPEC = {
    **{
        feature: tf.io.FixedLenFeature(shape=[1], dtype=tf.float32)
           for feature in _FEATURE_KEYS
       },
    _LABEL_KEY: tf.io.FixedLenFeature(shape=[1], dtype=tf.int64)
}


def _input_fn(file_pattern: List[str],
              data_accessor: tfx.components.DataAccessor,
              schema: schema_pb2.Schema,
              batch_size: int = 200) -> tf.data.Dataset:
  """Generates features and label for training.

  Args:
    file_pattern: List of paths or patterns of input tfrecord files.
    data_accessor: DataAccessor for converting input to RecordBatch.
    schema: schema of the input data.
    batch_size: representing the number of consecutive elements of returned
      dataset to combine in a single batch

  Returns:
    A dataset that contains (features, indices) tuple where features is a
      dictionary of Tensors, and indices is a single Tensor of label indices.
  """
  return data_accessor.tf_dataset_factory(
      file_pattern,
      tfxio.TensorFlowDatasetOptions(
          batch_size=batch_size, label_key=_LABEL_KEY),
      schema=schema).repeat()


def _build_keras_model() -> tf.keras.Model:
  """Creates a DNN Keras model for classifying penguin data.

  Returns:
    A Keras Model.
  """
  # The model below is built with Functional API, please refer to
  # https://www.tensorflow.org/guide/keras/overview for all API options.
  inputs = [keras.layers.Input(shape=(1,), name=f) for f in _FEATURE_KEYS]
  d = keras.layers.concatenate(inputs)
  for _ in range(2):
    d = keras.layers.Dense(8, activation='relu')(d)
  outputs = keras.layers.Dense(9)(d)

  model = keras.Model(inputs=inputs, outputs=outputs)
  model.compile(
      optimizer=keras.optimizers.Adam(1e-2),
      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
      metrics=[keras.metrics.SparseCategoricalAccuracy()])

  model.summary(print_fn=logging.info)
  return model


# TFX Trainer will call this function.
def run_fn(fn_args: tfx.components.FnArgs):
  """Train the model based on given args.

  Args:
    fn_args: Holds args used to train the model as name/value pairs.
  """

  # This schema is usually either an output of SchemaGen or a manually-curated
  # version provided by pipeline author. A schema can also derived from TFT
  # graph if a Transform component is used. In the case when either is missing,
  # `schema_from_feature_spec` could be used to generate schema from very simple
  # feature_spec, but the schema returned would be very primitive.
  schema = schema_utils.schema_from_feature_spec(_FEATURE_SPEC)

  train_dataset = _input_fn(
      fn_args.train_files,
      fn_args.data_accessor,
      schema,
      batch_size=_TRAIN_BATCH_SIZE)
  eval_dataset = _input_fn(
      fn_args.eval_files,
      fn_args.data_accessor,
      schema,
      batch_size=_EVAL_BATCH_SIZE)

  model = _build_keras_model()
  model.fit(
      train_dataset,
      steps_per_epoch=fn_args.train_steps,
      validation_data=eval_dataset,
      validation_steps=fn_args.eval_steps)
  print("model_dir", fn_args.serving_model_dir)

  # The result of the training should be saved in `fn_args.serving_model_dir`
  # directory.
  model.save(fn_args.serving_model_dir, save_format='tf')

Overwriting /home/mlops/project/test/penguin_trainer.py


In [15]:
from tfx.v1.proto import TrainArgs, EvalArgs
from tensorflow.python.keras.estimator import model_to_estimator

# model = tfdf.keras.RandomForestModel()


trainer = tfx.components.Trainer(
    module_file='/home/mlops/project/test/penguin_trainer.py',
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    train_args=TrainArgs(num_steps=100),
    eval_args=EvalArgs(num_steps=5)
)


In [16]:

context.run(trainer)

100/100 [==============================] - 1s 3ms/step - loss: 1.7016 - sparse_categorical_accuracy: 0.4200 - val_loss: 1.6804 - val_sparse_categorical_accuracy: 0.3800
model_dir /home/mlops/project/TFX/Trainer/model/851/Format-Serving
INFO:tensorflow:Assets written to: /home/mlops/project/TFX/Trainer/model/851/Format-Serving/assets


INFO:tensorflow:Assets written to: /home/mlops/project/TFX/Trainer/model/851/Format-Serving/assets


ExecutionResult(
    component_id: Trainer
    execution_id: 851
    outputs:
        model: Channel(
            type_name: Model
            artifacts: [Artifact(artifact: id: 496
        type_id: 23
        uri: "/home/mlops/project/TFX/Trainer/model/851"
        custom_properties {
          key: "name"
          value {
            string_value: "model"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Trainer"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.5.0"
          }
        }
        state: LIVE
        , artifact_type: id: 23
        name: "Model"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        model_run: Channel(
            type_name: ModelRun
            artifacts: [Artifact(artifact: id: 497
        type_id: 24
        uri: "/home/mlops/project/TFX/Trainer/model_run/851"
        custom_properties {
          key: "name"
          value {
            string_value: "model_run"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Trainer"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.5.0"
          }
        }
        state: LIVE
        , artifact_type: id: 24
        name: "ModelRun"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [17]:
model_artifact_dir = trainer.outputs['model'].get()[0].uri
pp.pprint(os.listdir(model_artifact_dir))
model_dir = os.path.join(model_artifact_dir, 'Format-Serving')
pp.pprint(os.listdir(model_dir))

['Format-Serving']
['assets', 'saved_model.pb', 'variables', 'keras_metadata.pb']


In [20]:
model_run_artifact_dir = trainer.outputs['model'].get()[0].uri

%load_ext tensorboard
%tensorboard --logdir {model_run_artifact_dir} --port=8008

Launching TensorBoard...

In [ ]:
%reload_ext tensorboard

In [19]:
import tfx
import tensorflow_model_analysis as tfma

model_resolver = tfx.v1.dsl.Resolver(
      strategy_class=tfx.v1.dsl.experimental.LatestBlessedModelStrategy,
      model=tfx.v1.dsl.Channel(type=tfx.types.standard_artifacts.Model),
      model_blessing=tfx.v1.dsl.Channel(
          type=tfx.types.standard_artifacts.ModelBlessing)).with_id(
              'latest_blessed_model_resolver')

# model_resolver.outputs['model']
context.run(model_resolver)
eval_config = tfma.EvalConfig(
      model_specs=[tfma.ModelSpec(label_key='spotify_genres_9')],
      slicing_specs=[
          # An empty slice spec means the overall slice, i.e. the whole dataset.
          tfma.SlicingSpec(),
          # Calculate metrics for each penguin species.
          tfma.SlicingSpec(feature_keys=['spotify_genres_9']),
          ],
      metrics_specs=[
          tfma.MetricsSpec(per_slice_thresholds={
              'sparse_categorical_accuracy':
                  tfma.PerSliceMetricThresholds(thresholds=[
                      tfma.PerSliceMetricThreshold(
                          slicing_specs=[tfma.SlicingSpec()],
                          threshold=tfma.MetricThreshold(
                              value_threshold=tfma.GenericValueThreshold(
                                   lower_bound={'value': 0.1}),
                              # Change threshold will be ignored if there is no
                              # baseline model resolved from MLMD (first run).
                              change_threshold=tfma.GenericChangeThreshold(
                                  direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                                  absolute={'value': -1e-10}))
                       )]),
          })],
      )
evaluator = tfx.components.Evaluator(
      examples=example_gen.outputs['examples'],
      model=trainer.outputs['model'],
      baseline_model=model_resolver.outputs['model'],
      eval_config=eval_config)

context.run(evaluator)

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


ExecutionResult(
    component_id: Evaluator
    execution_id: 853
    outputs:
        evaluation: Channel(
            type_name: ModelEvaluation
            artifacts: [Artifact(artifact: id: 498
        type_id: 27
        uri: "/home/mlops/project/TFX/Evaluator/evaluation/853"
        custom_properties {
          key: "name"
          value {
            string_value: "evaluation"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Evaluator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.5.0"
          }
        }
        state: LIVE
        , artifact_type: id: 27
        name: "ModelEvaluation"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        blessing: Channel(
            type_name: ModelBlessing
            artifacts: [Artifact(artifact: id: 499
        type_id: 28
        uri: "/home/mlops/project/TFX/Evaluator/blessing/853"
        custom_properties {
          key: "baseline_model"
          value {
            string_value: "/home/mlops/project/TFX/Trainer/model/666"
          }
        }
        custom_properties {
          key: "baseline_model_id"
          value {
            int_value: 332
          }
        }
        custom_properties {
          key: "blessed"
          value {
            int_value: 1
          }
        }
        custom_properties {
          key: "current_model"
          value {
            string_value: "/home/mlops/project/TFX/Trainer/model/851"
          }
        }
        custom_properties {
          key: "current_model_id"
          value {
            int_value: 496
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "blessing"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Evaluator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.5.0"
          }
        }
        state: LIVE
        , artifact_type: id: 28
        name: "ModelBlessing"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [21]:
eval_artifact = evaluator.outputs['evaluation']._artifacts[0]
eval_artifact

.type,<class 'tfx.types.standard_artifacts.ModelEvaluation'>
.uri,/home/mlops/project/TFX/Evaluator/evaluation/853


In [22]:
eval_result = tfma.load_eval_result(eval_artifact.uri)
tfma.view.render_slicing_metrics(eval_result, slicing_column='spotify_genres_9')

SlicingMetricsViewer(config={'weightedExamplesColumn': 'example_count'}, data=[{'slice': 'spotify_genres_9:3',…